<a href="https://colab.research.google.com/github/Vecheslav-Eremeev/Torch/blob/main/Pytorch_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imports

In [18]:
import warnings
warnings.filterwarnings('ignore')

In [19]:
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score

from keras.datasets import mnist

import torch
import torch.nn as nn

Load Data

In [3]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

11501568/11490434 [==============================] - 0s 0us/step


In [4]:
X_train.shape, y_train.shape

((60000, 28, 28), (60000,))

In [5]:
X_test.shape, y_test.shape

((10000, 28, 28), (10000,))

In [6]:
X_train = torch.from_numpy(np.array(X_train).reshape(-1,1,28,28)).float()
y_train = torch.from_numpy(np.array(y_train)).long()

In [7]:
train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True)

Train Model

In [8]:
class Image_class(nn.Module):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            nn.Conv2d(1, 28, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(28, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.Flatten(),
            nn.Linear(128 * 14 * 14, 128),
            nn.Dropout(0.3),
            nn.Linear(128, 10)
        )
    
    def forward(self, x):
        return self.network(x)

In [9]:
model = Image_class()
model

Image_class(
  (network): Sequential(
    (0): Conv2d(1, 28, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(28, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): Flatten(start_dim=1, end_dim=-1)
    (7): Linear(in_features=25088, out_features=128, bias=True)
    (8): Dropout(p=0.3, inplace=False)
    (9): Linear(in_features=128, out_features=10, bias=True)
  )
)

In [10]:
optimazer = torch.optim.Adam(model.parameters(), lr=1e-5)
loss = nn.CrossEntropyLoss()

In [11]:
if torch.cuda.is_available():
    model = model.cuda()

In [12]:
epochs = 40

In [13]:
for epoch in range(epochs):
    score = 0
    for batch in train_loader:
        X, y = batch
        if torch.cuda.is_available():
            X = X.cuda()
            y = y.cuda()
        model.train()
        out = model(X)
        y_loss = loss(out, y)
        model.zero_grad()
        y_loss.backward()
        optimazer.step()
        _, pred = torch.max(out, axis=1)
        score += (pred == y).sum()
    accuracy = score / len(X_train)
    print("{}/{} Epochs | Train Loss={:.4f} |Accuracy={:.4f}".format(epoch+1,epochs,y_loss,accuracy))

1/40 Epochs | Train Loss=0.2000 |Accuracy=0.8759
2/40 Epochs | Train Loss=0.2979 |Accuracy=0.9483
3/40 Epochs | Train Loss=0.0910 |Accuracy=0.9658
4/40 Epochs | Train Loss=0.0842 |Accuracy=0.9737
5/40 Epochs | Train Loss=0.0595 |Accuracy=0.9777
6/40 Epochs | Train Loss=0.0579 |Accuracy=0.9809
7/40 Epochs | Train Loss=0.0083 |Accuracy=0.9828
8/40 Epochs | Train Loss=0.0143 |Accuracy=0.9855
9/40 Epochs | Train Loss=0.0357 |Accuracy=0.9865
10/40 Epochs | Train Loss=0.0165 |Accuracy=0.9878
11/40 Epochs | Train Loss=0.0470 |Accuracy=0.9894
12/40 Epochs | Train Loss=0.0110 |Accuracy=0.9909
13/40 Epochs | Train Loss=0.0083 |Accuracy=0.9913
14/40 Epochs | Train Loss=0.0300 |Accuracy=0.9921
15/40 Epochs | Train Loss=0.1154 |Accuracy=0.9934
16/40 Epochs | Train Loss=0.0174 |Accuracy=0.9938
17/40 Epochs | Train Loss=0.0317 |Accuracy=0.9944
18/40 Epochs | Train Loss=0.0184 |Accuracy=0.9945
19/40 Epochs | Train Loss=0.0036 |Accuracy=0.9956
20/40 Epochs | Train Loss=0.0203 |Accuracy=0.9962
21/40 Epo

In [14]:
pred = []
for i in range(len(X_test)):
    img = X_test[i,:]
    img = img.reshape(1,1,28,28)
    img = torch.from_numpy(img).float()
    model.eval()
    if torch.cuda.is_available():
        img = img.cuda()
    output = model(img)
    _, predict = torch.max(output, 1)
    pred.append(predict.item())

In [16]:
accuracy_score(y_test, np.array(pred))

0.9879